In [ ]:
!pip install torch
!pip install transformers
!pip install datasets
!pip install scikit-learn
!pip install tqdm
!pip install pandas
!pip install plotly
!pip install kaleido
!pip install jupyter
!pip install matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 10.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires python-multipart, 

In [ ]:
from datasets import load_dataset

dataset = load_dataset("grammarly/detexd-benchmark", split='test')
print(dataset)
print(dataset.to_pandas().label.value_counts())

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/1023 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'annotator_1', 'annotator_2', 'annotator_3', 'label'],
    num_rows: 1023
})
0    687
1    336
Name: label, dtype: int64


In [ ]:
# Download hatebert models
# https://arxiv.org/pdf/2010.12472.pdf
# https://osf.io/tbd58/
!wget https://files.de-1.osf.io/v1/resources/tbd58/providers/osfstorage/?zip= -O hatebert.zip
!mkdir hatebert
!unzip hatebert.zip -d hatebert
!rm hatebert.zip

!unzip hatebert/HateBERT_fine_tuned_models/HateBERT_abuseval.zip -d hatebert/HateBERT_fine_tuned_models
!unzip hatebert/HateBERT_fine_tuned_models/HateBERT_hateval.zip -d hatebert/HateBERT_fine_tuned_models
!unzip hatebert/HateBERT_fine_tuned_models/HateBERT_offenseval.zip -d hatebert/HateBERT_fine_tuned_models
!rm hatebert/HateBERT_fine_tuned_models/HateBERT_abuseval.zip
!rm hatebert/HateBERT_fine_tuned_models/HateBERT_hateval.zip
!rm hatebert/HateBERT_fine_tuned_models/HateBERT_offenseval.zip

--2023-11-03 21:53:54--  https://files.de-1.osf.io/v1/resources/tbd58/providers/osfstorage/?zip=
Resolving files.de-1.osf.io (files.de-1.osf.io)... 35.186.249.111
Connecting to files.de-1.osf.io (files.de-1.osf.io)|35.186.249.111|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘hatebert.zip’

hatebert.zip            [  <=>               ]   2.72G  12.6MB/s    in 2m 1s   

2023-11-03 21:55:57 (23.0 MB/s) - ‘hatebert.zip’ saved [2920530719]

Archive:  hatebert.zip
  inflating: hatebert/README         
 extracting: hatebert/BERT_fine_tuned_models/BERT-hateval.zip  
 extracting: hatebert/BERT_fine_tuned_models/BERT-abuseval.zip  
 extracting: hatebert/BERT_fine_tuned_models/BERT-offenseval.zip  
 extracting: hatebert/HateBERT_fine_tuned_models/HateBERT_hateval.zip  
 extracting: hatebert/HateBERT_fine_tuned_models/HateBERT_abuseval.zip  
 extracting: hatebert/HateBERT_fine_tuned_models/HateBERT_offenseval.zip  
 extracting:

In [ ]:
from transformers import pipeline
from sklearn.metrics import precision_recall_fscore_support
from tqdm.auto import tqdm
from sklearn.metrics import precision_recall_curve, f1_score
import numpy as np
from transformers.pipelines.pt_utils import KeyDataset
import pandas as pd

metrics = []
for name in tqdm(['hatebert/HateBERT_fine_tuned_models/HateBERT_abuseval',
                  'hatebert/HateBERT_fine_tuned_models/HateBERT_hateval',
                  'hatebert/HateBERT_fine_tuned_models/HateBERT_offenseval']):
    pipe = pipeline("text-classification", model=name, device=0, batch_size=8)
    pipe.model.config.id2label = [0, 1]
    preds = tqdm(pipe(KeyDataset(dataset, 'text'), truncation=True, top_k=None), total=len(dataset))
    scores = np.array([next(p['score']
                            for p in pr if p['label'] == 1)
                       for pr in preds])

    precision, recall, thresholds = precision_recall_curve(dataset['label'], scores)
    f_scores = 2*(precision*recall)/(precision+recall)
    optimal_threshold_index = np.argmax(f_scores)
    optimal_threshold = thresholds[optimal_threshold_index]
    for tag, threshold in [('', 0.5), ('_opt', optimal_threshold)]:
        preds = scores > threshold
        metrics.append((name + tag,) + precision_recall_fscore_support(dataset['label'], preds, average='binary')[:-1])

metrics = pd.DataFrame(metrics, columns=['model', 'precision', 'recall', 'f1'])
metrics.model = metrics.model.str.split('/').str[-1]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1023 [00:00<?, ?it/s]

  0%|          | 0/1023 [00:00<?, ?it/s]

  0%|          | 0/1023 [00:00<?, ?it/s]

In [ ]:
metrics.style.format('{:.1%}', subset=['precision', 'recall', 'f1'])

,model,precision,recall,f1
0,HateBERT_abuseval,86.7%,11.6%,20.5%
1,HateBERT_abuseval_opt,57.0%,70.2%,62.9%
2,HateBERT_hateval,95.2%,6.0%,11.2%
3,HateBERT_hateval_opt,41.1%,86.0%,55.6%
4,HateBERT_offenseval,75.4%,31.0%,43.9%
5,HateBERT_offenseval_opt,60.1%,72.6%,65.8%


In [ ]:
# evaluate HateBERT on HatEval (Basile et al., 2019) (reported f-score was .645±.001)
# run this first: founta_basile_comparison.ipynb
import datasets

df = pd.read_csv('preds_basile.csv')
pipe = pipeline("text-classification", model='hatebert/HateBERT_fine_tuned_models/HateBERT_hateval', device=0, batch_size=8)
pipe.model.config.id2label = [0, 1]
preds = tqdm(pipe(KeyDataset(datasets.Dataset.from_pandas(df), 'text'), truncation=True), total=len(df))
preds = [p['label'] for p in preds]
df['hateval_pred'] = preds
df.to_csv('preds_basile.csv', index=False)
print(dict(zip(['precision', 'recall', 'f-score'],
               [f'{x:.1%}' for x in precision_recall_fscore_support(df.real, preds, average='binary')[:-1]])))

  0%|          | 0/2805 [00:00<?, ?it/s]

{'precision': '48.3%', 'recall': '96.4%', 'f-score': '64.3%'}


In [ ]:
# Note, there's a mismatch with the paper: 64.3% != .645
# possible reasons?
# 1) some floating-point fluctuations, different versions of torch, etc
# 2) two positive classes are treated separately